In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'amcess' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

sys.executable

'/home/edison/repo_gitlab/amcess/venv_amcess/bin/python'

In [2]:
from amcess.base_molecule import Molecule, Cluster

water = {
    "atoms": [
        ("O", 0, 0, 0),
        ("H", 0.58708, 0.75754, 0),
        ("H", -0.58708, 0.75754, 0),
    ],
    "charge": 0,
    "multiplicity": 1,
}

water_dimer = Cluster(water, water)
water_dimer.sphere_radius = 1.5
water_dimer.sphere_center = (0, 0, 0)

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


In [3]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 
# 3Dmol.js viewer in a notebook.
!pip install py3Dmol
import py3Dmol

def visualize(
        xyz_file: str, 
        sphere_radius: int, 
        sphere_center: tuple,
        style='sphere',
        ):
        """
        Visualize the molecule with a sphere around the molecule.
        """

        sr = sphere_radius
        cx, cy, cz = sphere_center

        if not xyz_file:
                return None

        # starting visualization
        xyz_view = py3Dmol.view() # width=400, height=300)#, linked=False, viewergrid=(2,2))
        xyz_view.addModelsAsFrames(xyz_file,'xyz')
        # xyz_view.setStyle({style: {'radius': 0.6}})

        if style == 'stick':
                xyz_view.setStyle({'stick': {'radius': 0.15}})
        else:
                xyz_view.setStyle({'sphere': {'radius': 0.8}})


        xyz_view.zoomTo()

        # animation base on several XYZ coordinates snapshots
        # xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})
        xyz_view.animate({'loop': 'backAndForth', 'speed': 1})


        # sphere center
        sc = {"x": cx, "y": cy, "z": cz}

        # 20% to check visually that everything is inside
        xyz_view.addSphere({'center': sc, 
                'radius': sr, 
                'color' :'yellow',
                'alpha': 0.5,
                })

        # cartesian 3D axes
        x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
        y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
        z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

        xyz_view.addLine(x_axis)
        xyz_view.addLine(y_axis)
        xyz_view.addLine(z_axis)

        xyz_view.addLabel("x", {
                'position':x_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("y", {
                'position':y_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("z", {
                'position':z_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })

        return xyz_view

In [4]:
# importing de Molecule Class
from amcess.base_molecule import Molecule, Cluster

water = {
    "atoms": [
        ("O", 0, 0, 0),
        ("H", 0.58708, 0.75754, 0),
        ("H", -0.58708, 0.75754, 0),
    ],
    "charge": 0,
    "multiplicity": 1,
}

water_dimer = Cluster(water, water)
water_dimer.sphere_radius = 1.5
water_dimer.sphere_center = (0, 0, 0)

# avoiding atomic overlapping
water_dimer = water_dimer.initialize_cluster(max_closeness=0.5)


In [5]:
%%time

from amcess.search_engine import simulated_annealing

simulation = simulated_annealing(
    cluster_settings={
        "cluster": water_dimer,
        "max_closeness": 0.5,
        "max_step": 0.6,
        "max_rotation": 30,
    },
    energy_settings={
        "energy": "pyscf",
        "hamiltonian": "b3lyp",
        "basis": "aug-cc-pvdz",
        "cpu": 4,
    },
    annealing_settings={
        "file_name": "water_dimer",
        "criterion": "metropolis",  # 'metropolis' or 'delta_energy'
        "temperature_grid": "geometric",
        "max_cycle": 1000,
    },
    temperature_settings={
        "initial_temperature": 100.0,
        "final_temperature": 700.0,
        "total_temperatures": 50,
    },
)

print(simulation)

#--------------------------------------------------------------------------------
Simulated Annealing:
	Cooling progression = geometric
	Initial temperature= 100.0 K
	Final temperature= 700.0 K
	Total number of temperatures = 50 points
	Acceptance criterion = metropolis
	Maximum search per temperature = 1000
#--------------------------------------------------------------------------------
Molecular Cluster:
	Output file name (prefix) = water_dimer
	Lowest energy configuration file = water_dimer_lowest_energy.xyz
	Accepted configuration file = water_dimer_configuration.xyz
	Energy data file = water_dimer_energy.dat
	Sphere radius = 1.5
	Sphere center = (0, 0, 0)
	Maximum translation = 0.6
	Maximum rotation = 30
	Maximum closeness between molecules = 0.5
	Number of molecules = 2
	Number of atoms = 6
#--------------------------------------------------------------------------------
Energy function:
	energy = pyscf
	hamiltonian = b3lyp
	basis = aug-cc-pvdz
	cpu = 4
#------------------------

In [6]:
xyz_file = open("water_dimer_configuration.xyz", "r").read()

view = visualize(        
    xyz_file = xyz_file,
    sphere_radius = water_dimer.sphere_radius * 1.5,
    sphere_center = water_dimer.sphere_center,
    style="stick",
)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
w4 = Cluster(2 * water_dimer)
w4.sphere_radius = 2.0
w4.sphere_center = (0, 0, 0)

# avoiding atomic overlapping
w4 = w4.initialize_cluster(max_closeness=1.0)

In [8]:
%%time

from amcess.search_engine import simulated_annealing

simulation = simulated_annealing(
    cluster_settings={
        "cluster": w4,
        "max_closeness": 0.5,
        "max_step": 0.6,
        "max_rotation": 30,
    },
    energy_settings={
        "energy": "pyscf",
        "hamiltonian": "b3lyp",
        "basis": "aug-cc-pvdz",
        "cpu": 4,
    },
    annealing_settings={
        "criterion": "metropolis",  # 'metropolis' or 'delta_energy'
        "file_name": "w4",
        "temperature_grid": "geometric",
        "max_cycle": 1000,
    },
    temperature_settings={
        "initial_temperature": 100.0,
        "final_temperature": 700.0,
        "total_temperatures": 100,
    },
)

print(simulation)

#--------------------------------------------------------------------------------
Simulated Annealing:
	Cooling progression = geometric
	Initial temperature= 100.0 K
	Final temperature= 700.0 K
	Total number of temperatures = 100 points
	Acceptance criterion = metropolis
	Maximum search per temperature = 1000
#--------------------------------------------------------------------------------
Molecular Cluster:
	Output file name (prefix) = w4
	Lowest energy configuration file = w4_lowest_energy.xyz
	Accepted configuration file = w4_configuration.xyz
	Energy data file = w4_energy.dat
	Sphere radius = 2.0
	Sphere center = (0, 0, 0)
	Maximum translation = 0.6
	Maximum rotation = 30
	Maximum closeness between molecules = 0.5
	Number of molecules = 4
	Number of atoms = 12
#--------------------------------------------------------------------------------
Energy function:
	energy = pyscf
	hamiltonian = b3lyp
	basis = aug-cc-pvdz
	cpu = 4
#----------------------------------------------------------

In [9]:
xyz_file = open("w4_configuration.xyz", "r").read()

view = visualize(        
    xyz_file = xyz_file,
    sphere_radius = w4.sphere_radius * 1.2,
    sphere_center = w4.sphere_center,
    style="stick",
)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol